# A tutorial for Scikit-Recommender

## Download and extract dataset

In [1]:
from skrec.io import MovieLens100k

data_path = MovieLens100k.download_and_extract("./dataset")

ModuleNotFoundError: No module named 'skrec'

## Pre-process and save dataset

In [ ]:
from skrec.io import Preprocessor

processor = Preprocessor()
processor.load_data(data_path, sep="\t", columns="UIRT")
processor.drop_duplicates(keep="last")  # remove duplicate interactions
processor.filter_data(user_min=5, item_min=5)  # filter users and items with few interactions
processor.remap_data_id()  # remap IDs of users and items
# data.split_data_by_leave_out(valid=1, test=1)
processor.split_data_by_ratio(train=0.7, valid=0.0, test=0.3, by_time=True)  # split dataset
data_dir = processor.save_data()# save to disk

## Read config

In [ ]:
from typing import Union, Tuple, List
from skrec import Config

class RunConfig(Config):
    def __init__(self,
                 recommender="BPRMF",
                 data_dir="",
                 file_column="UIRT", column_sep='\t',
                 gpu_id=0, gpu_mem=0.99,
                 metric=("Precision", "Recall", "MAP", "NDCG", "MRR"),
                 top_k=(10, 20, 30, 40, 50, 100),
                 test_batch_size=64,
                 test_thread=4,
                 seed=2021,
                 **kwargs):
        super(RunConfig, self).__init__(**kwargs)
        self.recommender: str = recommender
        self.data_dir: str = data_dir
        self.file_column: str = file_column
        self.sep: str = column_sep
        self.gpu_id = gpu_id
        self.gpu_mem = gpu_mem
        self.metric: Union[None, str, Tuple[str], List[str]] = metric
        self.top_k: Union[int, List[int], Tuple[int]] = top_k
        self.test_batch_size: int = test_batch_size
        self.test_thread: int = test_thread
        self.seed = seed

    def _validate(self):
        assert isinstance(self.recommender, str)
        assert isinstance(self.data_dir, str)
        assert isinstance(self.file_column, str)
        assert isinstance(self.sep, str)
        assert isinstance(self.test_batch_size, int) and self.test_batch_size > 0
        assert isinstance(self.test_thread, int) and self.test_thread > 0
        assert isinstance(self.seed, int) and self.seed >= 0

In [ ]:
run_config = RunConfig()
run_config.parse_args_from_ini("./skrec.ini")  # parse args from file and overwrite the default values
# run_config.parse_args_from_cmd()  # parse args from cmd and overwrite the previous values

## Load dataset

In [ ]:
from skrec import Dataset

dataset = Dataset(run_config.data_dir, run_config.sep, run_config.file_column)

## Create evaluator

In [ ]:
from skrec import RankingEvaluator

evaluator = RankingEvaluator(dataset.train_data.to_user_dict(),
                                 dataset.test_data.to_user_dict(),
                                 metric=run_config.metric, top_k=run_config.top_k,
                                 batch_size=run_config.test_batch_size,
                                 num_thread=run_config.test_thread)

## Create model

In [ ]:
from skrec.recommender.BPRMF import BPRMF, BPRMFConfig

bpr_config = BPRMFConfig(epochs=50)
bpr_config.parse_args_from_ini("./skrec.ini")  # , "bpr"

bpr_model = BPRMF(dataset, bpr_config, evaluator)

## Train model

In [ ]:
bpr_model.fit()